In [50]:
# Import the requests library.
import requests
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [51]:
# Read NYT data into dataframe
nyt_data = pd.DataFrame(pd.read_csv('us-counties.csv'))
nyt_data

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1891340,2021-11-06,Sweetwater,Wyoming,56037.0,7695,91.0
1891341,2021-11-06,Teton,Wyoming,56039.0,5243,14.0
1891342,2021-11-06,Uinta,Wyoming,56041.0,3901,27.0
1891343,2021-11-06,Washakie,Wyoming,56043.0,1780,35.0


In [52]:
# Convert the date column in the NYT data to a datetime object using pd.to_datetime
nyt_data['date'] = pd.to_datetime(nyt_data['date'])
nyt_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [53]:
# 1. Remove the data for dates before 7-31-20 and after 10-22-21
# 2. Bin the date column by weeks ending every friday
# 3. Sum the cases and deaths columns for each week binned

# Step 1:
nyt_data_filtered = nyt_data[nyt_data['date'] >= '2020-07-25']
nyt_data_filtered = nyt_data_filtered[nyt_data_filtered['date'] <= '2021-10-22']
nyt_data_filtered

,date,county,state,fips,cases,deaths
366652,2020-07-25,Autauga,Alabama,1001.0,932,21.0
366653,2020-07-25,Baldwin,Alabama,1003.0,2662,18.0
366654,2020-07-25,Barbour,Alabama,1005.0,552,4.0
366655,2020-07-25,Bibb,Alabama,1007.0,318,2.0
366656,2020-07-25,Blount,Alabama,1009.0,637,1.0
...,...,...,...,...,...,...
1842600,2021-10-22,Sweetwater,Wyoming,56037.0,7414,79.0
1842601,2021-10-22,Teton,Wyoming,56039.0,5119,14.0
1842602,2021-10-22,Uinta,Wyoming,56041.0,3781,26.0
1842603,2021-10-22,Washakie,Wyoming,56043.0,1676,33.0


In [54]:
# Step 2:
# Step 3:

nyt_data_grouped = nyt_data_filtered.groupby(['fips', pd.Grouper(key='date', freq='W-FRI')]).agg({'cases':'sum','deaths':'sum'}).reset_index()
nyt_data_grouped

,fips,date,cases,deaths
0,1001.0,2020-07-31,6817,147.0
1,1001.0,2020-08-07,7503,150.0
2,1001.0,2020-08-14,8531,158.0
3,1001.0,2020-08-21,9009,161.0
4,1001.0,2020-08-28,9484,161.0
...,...,...,...,...
208977,78030.0,2021-09-24,24720,297.0
208978,78030.0,2021-10-01,24982,304.0
208979,78030.0,2021-10-08,25204,308.0
208980,78030.0,2021-10-15,25290,311.0


In [55]:
# Set each column to a string value so we can concatonate them together 
nyt_data_grouped['fips'] = nyt_data_grouped['fips'].astype('str')
nyt_data_grouped['date'] = nyt_data_grouped['date'].astype('str')
nyt_data_grouped['fips'].dtype

dtype('O')

In [56]:
# Concatonate the date column and the fips column to create fips_date
nyt_data_grouped['fips_date'] = nyt_data_grouped['fips'] + nyt_data_grouped['date']
nyt_data_grouped

,fips,date,cases,deaths,fips_date
0,1001.0,2020-07-31,6817,147.0,1001.02020-07-31
1,1001.0,2020-08-07,7503,150.0,1001.02020-08-07
2,1001.0,2020-08-14,8531,158.0,1001.02020-08-14
3,1001.0,2020-08-21,9009,161.0,1001.02020-08-21
4,1001.0,2020-08-28,9484,161.0,1001.02020-08-28
...,...,...,...,...,...
208977,78030.0,2021-09-24,24720,297.0,78030.02021-09-24
208978,78030.0,2021-10-01,24982,304.0,78030.02021-10-01
208979,78030.0,2021-10-08,25204,308.0,78030.02021-10-08
208980,78030.0,2021-10-15,25290,311.0,78030.02021-10-15


In [57]:
# Read Hospital data (healthdata.gov) into dataframe
hospital_data_df = pd.DataFrame(pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv'))

In [58]:
hospital_data_df

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_7_day_coverage,previous_day_admission_adult_covid_suspected_7_day_coverage,previous_day_admission_pediatric_covid_suspected_7_day_coverage,previous_week_personnel_covid_vaccinated_doses_administered_7_day,total_personnel_covid_vaccinated_doses_none_7_day,total_personnel_covid_vaccinated_doses_one_7_day,total_personnel_covid_vaccinated_doses_all_7_day,previous_week_patients_covid_vaccinated_doses_one_7_day,previous_week_patients_covid_vaccinated_doses_all_7_day,is_corrected
0,140158,2021/10/15,IL,140158,INSIGHT HOSPITAL AND MEDICAL CENTER CHICAGO,2525 S MICHIGAN AVE,CHICAGO,60616.0,Short Term,17031.0,...,7,7,7,0.0,NaN,NaN,NaN,0.0,NaN,False
1,450162,2021/10/15,TX,450162,GRACE SURGICAL HOSPITAL,2412 50TH ST,LUBBOCK,79412.0,Short Term,48303.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
2,451318,2021/10/15,TX,451318,STONEWALL MEMORIAL HOSPITAL DISTRICT,821 NORTH BROADWAY,ASPERMONT,79502.0,Critical Access Hospitals,48433.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
3,050769,2021/10/08,CA,050769,HOAG ORTHOPEDIC INSTITUTE,16250 SAND CANYON AVENUE,IRVINE,92618.0,Short Term,6059.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
4,291500,2021/10/08,NV,291500,NATHAN ADELSON HOSPICE,4141 UNIVERSITY CENTER DR,LAS VEGAS,89119.0,Short Term,32003.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324743,100291,2020/08/07,FL,100291,STEWARD REGIONAL MEDICAL CENTER,250 NORTH WICKHAM ROAD,MELBOURNE,32935.0,Short Term,12009.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
324744,250096,2020/07/31,MS,250096,MERIT HEALTH RANKIN,350 CROSSGATES BLVD,BRANDON,39042.0,Short Term,28121.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
324745,241345,2020/07/31,MN,241345,MAYO CLINIC HEALTH SYSTEM-WASECA,501 NORTH STATE STREET,WASECA,56093.0,Critical Access Hospitals,27161.0,...,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,False
324746,271311,2020/08/07,MT,271311,BIG SANDY MEDICAL CENTER,166 MONTANA AVE E,BIG SANDY,59520.0,Critical Access Hospitals,30015.0,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,False


In [59]:
# Create list of columns that have string or floating point decimal data types
drop_list_hospitals = ['hospital_pk', 'state', 'ccn', 'hospital_name', 'address', 'city', 'zip', 'hospital_subtype', 
                       'is_metro_micro', 'total_beds_7_day_avg', 'all_adult_hospital_beds_7_day_avg', 
                       'all_adult_hospital_inpatient_beds_7_day_avg', 'inpatient_beds_used_7_day_avg', 
                       'all_adult_hospital_inpatient_bed_occupied_7_day_avg', 
                       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg', 
                       'total_adult_patients_hospitalized_confirmed_covid_7_day_avg', 
                       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg', 
                       'total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg', 'inpatient_beds_7_day_avg', 
                       'total_icu_beds_7_day_avg', 'total_staffed_adult_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg', 
                       'staffed_adult_icu_bed_occupancy_7_day_avg', 
                       'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg', 
                       'staffed_icu_adult_patients_confirmed_covid_7_day_avg', 
                       'total_patients_hospitalized_confirmed_influenza_7_day_avg', 
                       'icu_patients_confirmed_influenza_7_day_avg', 
                       'total_patients_hospitalized_confirmed_influenza_and_covid_7_day_avg', 
                       'all_adult_hospital_inpatient_beds_7_day_coverage', 
                       'inpatient_beds_used_7_day_coverage', 'all_adult_hospital_inpatient_bed_occupied_7_day_coverage', 
                       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage', 
                       'total_adult_patients_hospitalized_confirmed_covid_7_day_coverage', 
                       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage', 
                       'total_pediatric_patients_hospitalized_confirmed_covid_7_day_coverage', 
                       'inpatient_beds_7_day_coverage', 'total_icu_beds_7_day_coverage', 
                       'total_staffed_adult_icu_beds_7_day_coverage', 'icu_beds_used_7_day_coverage', 
                       'staffed_adult_icu_bed_occupancy_7_day_coverage', 
                       'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_coverage', 
                       'staffed_icu_adult_patients_confirmed_covid_7_day_coverage', 
                       'total_patients_hospitalized_confirmed_influenza_7_day_coverage', 
                       'icu_patients_confirmed_influenza_7_day_coverage', 
                       'total_patients_hospitalized_confirmed_influenza_and_covid_7_day_coverage', 
                       'previous_day_admission_adult_covid_confirmed_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_18-19_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_20-29_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_30-39_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_40-49_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_50-59_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_60-69_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_70-79_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_80+_7_day_sum', 
                       'previous_day_admission_adult_covid_confirmed_unknown_7_day_sum', 
                       'previous_day_admission_pediatric_covid_confirmed_7_day_sum', 
                       'previous_day_covid_ED_visits_7_day_sum', 'previous_day_admission_adult_covid_suspected_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_18-19_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_20-29_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_30-39_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_40-49_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_50-59_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_60-69_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_70-79_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_80+_7_day_sum', 
                       'previous_day_admission_adult_covid_suspected_unknown_7_day_sum', 
                       'previous_day_admission_pediatric_covid_suspected_7_day_sum', 
                       'previous_day_total_ED_visits_7_day_sum', 'previous_day_admission_influenza_confirmed_7_day_sum', 
                       'geocoded_hospital_address', 'hhs_ids', 
                       'previous_day_admission_adult_covid_confirmed_7_day_coverage', 
                       'previous_day_admission_pediatric_covid_confirmed_7_day_coverage', 
                       'previous_day_admission_adult_covid_suspected_7_day_coverage', 
                       'previous_day_admission_pediatric_covid_suspected_7_day_coverage', 
                       'previous_week_personnel_covid_vaccinated_doses_administered_7_day', 
                       'total_personnel_covid_vaccinated_doses_none_7_day', 
                       'total_personnel_covid_vaccinated_doses_one_7_day', 
                       'total_personnel_covid_vaccinated_doses_all_7_day', 
                       'previous_week_patients_covid_vaccinated_doses_one_7_day', 
                       'previous_week_patients_covid_vaccinated_doses_all_7_day', 'is_corrected', 
                       'total_beds_7_day_coverage', 'all_adult_hospital_beds_7_day_coverage',
                       'total_patients_hospitalized_confirmed_influenza_7_day_sum', 
                       'icu_patients_confirmed_influenza_7_day_sum', 
                       'total_patients_hospitalized_confirmed_influenza_and_covid_7_day_sum'
]

In [60]:
# Remove drop_list columns from df
hospital_data_df.drop(drop_list_hospitals, axis=1, inplace=True)
hospital_data_df.head()

,collection_week,fips_code,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
0,2021/10/15,17031.0,350.0,350.0,252.0,160.0,160.0,7.0,7.0,0.0,0.0,252.0,28.0,28.0,16.0,16.0,7.0,7.0
1,2021/10/15,48303.0,268.0,268.0,261.0,34.0,34.0,0.0,0.0,0.0,0.0,261.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021/10/15,48433.0,77.0,77.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021/10/08,6059.0,245.0,245.0,133.0,126.0,126.0,0.0,0.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021/10/08,32003.0,266.0,266.0,266.0,191.0,191.0,11.0,11.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# Convert the date column in the hospital data to a datetime object using pd.to_datetime
hospital_data_df['collection_week'] = pd.to_datetime(hospital_data_df['collection_week'])
# Check to verify the date is now a datetime object
hospital_data_df.dtypes

collection_week                                                                  datetime64[ns]
fips_code                                                                               float64
total_beds_7_day_sum                                                                    float64
all_adult_hospital_beds_7_day_sum                                                       float64
all_adult_hospital_inpatient_beds_7_day_sum                                             float64
inpatient_beds_used_7_day_sum                                                           float64
all_adult_hospital_inpatient_bed_occupied_7_day_sum                                     float64
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum               float64
total_adult_patients_hospitalized_confirmed_covid_7_day_sum                             float64
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum           float64
total_pediatric_patients_hospitalized_co

In [62]:
# Sort the hospital data by date
hospital_data_df = hospital_data_df.sort_values(by='collection_week')
hospital_data_df

# The hospital data is binned by collection week ending on every friday

,collection_week,fips_code,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
324747,2020-07-31,13013.0,266.0,NaN,NaN,199.0,NaN,0.0,NaN,-999999.0,NaN,266.0,0.0,NaN,0.0,NaN,NaN,NaN
319353,2020-07-31,12099.0,3332.0,2710.0,2710.0,1873.0,1859.0,266.0,266.0,0.0,0.0,3206.0,287.0,287.0,178.0,178.0,41.0,41.0
319356,2020-07-31,19049.0,102.0,126.0,84.0,-999999.0,9.0,4.0,-999999.0,0.0,0.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0
319357,2020-07-31,6037.0,824.0,404.0,341.0,296.0,296.0,5.0,0.0,0.0,0.0,341.0,20.0,20.0,18.0,18.0,0.0,0.0
319358,2020-07-31,39095.0,1476.0,NaN,NaN,716.0,NaN,52.0,52.0,0.0,0.0,1446.0,132.0,NaN,94.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4644,2021-10-22,21231.0,175.0,79.0,100.0,79.0,79.0,-999999.0,-999999.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
4645,2021-10-22,21111.0,12047.0,7935.0,6736.0,7319.0,6245.0,1071.0,422.0,257.0,42.0,7897.0,1091.0,804.0,695.0,695.0,185.0,107.0
9753,2021-10-22,48303.0,965.0,358.0,274.0,714.0,241.0,7.0,7.0,14.0,14.0,881.0,164.0,26.0,79.0,7.0,-999999.0,-999999.0
9755,2021-10-22,5009.0,623.0,560.0,364.0,240.0,227.0,6.0,6.0,0.0,0.0,448.0,84.0,84.0,26.0,26.0,5.0,5.0


In [63]:
# Group and sort hospital_data_df by fips_code
hospital_data_df = hospital_data_df.groupby(['fips_code', pd.Grouper(key='collection_week', freq='W-FRI')]).agg({'total_beds_7_day_sum':'sum','all_adult_hospital_beds_7_day_sum':'sum', 'all_adult_hospital_inpatient_beds_7_day_sum':'sum', 'inpatient_beds_used_7_day_sum':'sum', 'all_adult_hospital_inpatient_bed_occupied_7_day_sum':'sum', 
                                                                                                                       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum':'sum', 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum': 'sum', 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum':'sum', 'total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum':'sum', 'inpatient_beds_7_day_sum':'sum', 'total_icu_beds_7_day_sum':'sum', 
                                                                                                                       'total_staffed_adult_icu_beds_7_day_sum':'sum', 'icu_beds_used_7_day_sum':'sum', 'staffed_adult_icu_bed_occupancy_7_day_sum':'sum', 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum':'sum', 'staffed_icu_adult_patients_confirmed_covid_7_day_sum':'sum'}).reset_index()
hospital_data_df

,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
0,1001.0,2020-07-31,574.0,574.0,455.0,363.0,363.0,117.0,103.0,-999999.0,0.0,455.0,42.0,42.0,40.0,40.0,28.0,26.0
1,1001.0,2020-08-07,574.0,574.0,455.0,369.0,369.0,153.0,139.0,-999999.0,-999999.0,455.0,42.0,42.0,41.0,41.0,38.0,37.0
2,1001.0,2020-08-14,574.0,574.0,455.0,384.0,384.0,125.0,111.0,0.0,-999999.0,455.0,42.0,42.0,42.0,42.0,38.0,32.0
3,1001.0,2020-08-21,574.0,574.0,455.0,348.0,348.0,86.0,74.0,0.0,0.0,455.0,42.0,42.0,38.0,38.0,48.0,35.0
4,1001.0,2020-08-28,574.0,574.0,455.0,346.0,346.0,77.0,75.0,0.0,0.0,455.0,42.0,42.0,40.0,40.0,18.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159866,78020.0,2021-09-24,924.0,868.0,728.0,410.0,389.0,20.0,20.0,0.0,0.0,784.0,56.0,56.0,29.0,29.0,7.0,7.0
159867,78020.0,2021-10-01,924.0,868.0,728.0,362.0,340.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,43.0,43.0,7.0,7.0
159868,78020.0,2021-10-08,924.0,868.0,728.0,412.0,381.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,48.0,48.0,7.0,7.0
159869,78020.0,2021-10-15,924.0,868.0,728.0,402.0,365.0,7.0,7.0,0.0,0.0,784.0,56.0,56.0,40.0,40.0,7.0,7.0


In [64]:
# Convert the hospital date and fips_code column to strings
hospital_data_df['collection_week'] = hospital_data_df['collection_week'].astype('str')
hospital_data_df['fips_code'] = hospital_data_df['fips_code'].astype('str')

In [65]:
# Concat the date and fips_code for new columns fips_date
hospital_data_df['fips_date'] = hospital_data_df['fips_code'] + hospital_data_df['collection_week']
hospital_data_df

,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum,fips_date
0,1001.0,2020-07-31,574.0,574.0,455.0,363.0,363.0,117.0,103.0,-999999.0,0.0,455.0,42.0,42.0,40.0,40.0,28.0,26.0,1001.02020-07-31
1,1001.0,2020-08-07,574.0,574.0,455.0,369.0,369.0,153.0,139.0,-999999.0,-999999.0,455.0,42.0,42.0,41.0,41.0,38.0,37.0,1001.02020-08-07
2,1001.0,2020-08-14,574.0,574.0,455.0,384.0,384.0,125.0,111.0,0.0,-999999.0,455.0,42.0,42.0,42.0,42.0,38.0,32.0,1001.02020-08-14
3,1001.0,2020-08-21,574.0,574.0,455.0,348.0,348.0,86.0,74.0,0.0,0.0,455.0,42.0,42.0,38.0,38.0,48.0,35.0,1001.02020-08-21
4,1001.0,2020-08-28,574.0,574.0,455.0,346.0,346.0,77.0,75.0,0.0,0.0,455.0,42.0,42.0,40.0,40.0,18.0,18.0,1001.02020-08-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159866,78020.0,2021-09-24,924.0,868.0,728.0,410.0,389.0,20.0,20.0,0.0,0.0,784.0,56.0,56.0,29.0,29.0,7.0,7.0,78020.02021-09-24
159867,78020.0,2021-10-01,924.0,868.0,728.0,362.0,340.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,43.0,43.0,7.0,7.0,78020.02021-10-01
159868,78020.0,2021-10-08,924.0,868.0,728.0,412.0,381.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,48.0,48.0,7.0,7.0,78020.02021-10-08
159869,78020.0,2021-10-15,924.0,868.0,728.0,402.0,365.0,7.0,7.0,0.0,0.0,784.0,56.0,56.0,40.0,40.0,7.0,7.0,78020.02021-10-15


In [66]:
hospital_data_df.dtypes

fips_code                                                                         object
collection_week                                                                   object
total_beds_7_day_sum                                                             float64
all_adult_hospital_beds_7_day_sum                                                float64
all_adult_hospital_inpatient_beds_7_day_sum                                      float64
inpatient_beds_used_7_day_sum                                                    float64
all_adult_hospital_inpatient_bed_occupied_7_day_sum                              float64
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum        float64
total_adult_patients_hospitalized_confirmed_covid_7_day_sum                      float64
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum    float64
total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum                  float64
inpatient_beds_7_day_

In [67]:
# Convert all negative values to 0
num = hospital_data_df._get_numeric_data()
num[num < 0] = 0

In [68]:
hospital_data_df

,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum,fips_date
0,1001.0,2020-07-31,574.0,574.0,455.0,363.0,363.0,117.0,103.0,0.0,0.0,455.0,42.0,42.0,40.0,40.0,28.0,26.0,1001.02020-07-31
1,1001.0,2020-08-07,574.0,574.0,455.0,369.0,369.0,153.0,139.0,0.0,0.0,455.0,42.0,42.0,41.0,41.0,38.0,37.0,1001.02020-08-07
2,1001.0,2020-08-14,574.0,574.0,455.0,384.0,384.0,125.0,111.0,0.0,0.0,455.0,42.0,42.0,42.0,42.0,38.0,32.0,1001.02020-08-14
3,1001.0,2020-08-21,574.0,574.0,455.0,348.0,348.0,86.0,74.0,0.0,0.0,455.0,42.0,42.0,38.0,38.0,48.0,35.0,1001.02020-08-21
4,1001.0,2020-08-28,574.0,574.0,455.0,346.0,346.0,77.0,75.0,0.0,0.0,455.0,42.0,42.0,40.0,40.0,18.0,18.0,1001.02020-08-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159866,78020.0,2021-09-24,924.0,868.0,728.0,410.0,389.0,20.0,20.0,0.0,0.0,784.0,56.0,56.0,29.0,29.0,7.0,7.0,78020.02021-09-24
159867,78020.0,2021-10-01,924.0,868.0,728.0,362.0,340.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,43.0,43.0,7.0,7.0,78020.02021-10-01
159868,78020.0,2021-10-08,924.0,868.0,728.0,412.0,381.0,12.0,12.0,0.0,0.0,784.0,56.0,56.0,48.0,48.0,7.0,7.0,78020.02021-10-08
159869,78020.0,2021-10-15,924.0,868.0,728.0,402.0,365.0,7.0,7.0,0.0,0.0,784.0,56.0,56.0,40.0,40.0,7.0,7.0,78020.02021-10-15


In [69]:
# Create a new df to groupby the new fips_date column so each row is unique
hospital_data_df2 = hospital_data_df.groupby(['fips_date']).agg({'total_beds_7_day_sum':'sum', 
                                                                 'all_adult_hospital_beds_7_day_sum':'sum', 
                                                                 'all_adult_hospital_inpatient_beds_7_day_sum':'sum', 
                                                                 'inpatient_beds_used_7_day_sum':'sum', 
                                                                 'all_adult_hospital_inpatient_bed_occupied_7_day_sum':'sum',
                                                                 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum':'sum', 
                                                                 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum': 'sum', 
                                                                 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum':'sum', 
                                                                 'total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum':'sum', 
                                                                 'inpatient_beds_7_day_sum':'sum', 'total_icu_beds_7_day_sum':'sum', 
                                                                 'total_staffed_adult_icu_beds_7_day_sum':'sum', 'icu_beds_used_7_day_sum':'sum', 
                                                                 'staffed_adult_icu_bed_occupancy_7_day_sum':'sum', 
                                                                 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum':'sum', 
                                                                 'staffed_icu_adult_patients_confirmed_covid_7_day_sum':'sum'}).reset_index()
hospital_data_df2

,fips_date,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
0,10001.02020-07-31,2100.0,1939.0,1688.0,1713.0,1558.0,152.0,84.0,0.0,0.0,1876.0,322.0,203.0,266.0,189.0,28.0,28.0
1,10001.02020-08-07,2100.0,1939.0,1694.0,1672.0,1548.0,112.0,71.0,0.0,0.0,1876.0,322.0,203.0,273.0,188.0,22.0,22.0
2,10001.02020-08-14,2100.0,1939.0,1210.0,1141.0,1028.0,132.0,55.0,0.0,0.0,1340.0,198.0,134.0,183.0,128.0,29.0,29.0
3,10001.02020-08-21,2100.0,1939.0,1729.0,1650.0,1502.0,93.0,57.0,0.0,0.0,1908.0,285.0,193.0,245.0,171.0,23.0,23.0
4,10001.02020-08-28,2100.0,1939.0,1939.0,1730.0,1624.0,65.0,49.0,0.0,0.0,2100.0,315.0,203.0,251.0,182.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159866,9015.02021-09-24,951.0,937.0,783.0,565.0,565.0,56.0,42.0,0.0,0.0,783.0,119.0,119.0,34.0,34.0,14.0,10.0
159867,9015.02021-10-01,956.0,942.0,788.0,530.0,530.0,61.0,57.0,0.0,0.0,788.0,119.0,119.0,32.0,32.0,15.0,14.0
159868,9015.02021-10-08,956.0,942.0,788.0,557.0,557.0,87.0,85.0,0.0,0.0,788.0,119.0,119.0,32.0,32.0,20.0,20.0
159869,9015.02021-10-15,955.0,941.0,787.0,521.0,521.0,76.0,74.0,0.0,0.0,787.0,119.0,119.0,26.0,26.0,19.0,19.0


In [70]:
f = hospital_data_df['fips_date'].unique()


In [71]:
x = nyt_data_grouped['fips_date'].unique()

In [72]:
# Create list of fips_date values that only appear in one data set

remove_list = list(set(x).symmetric_difference(set(f)))
remove_list

['13011.02021-04-16',
 '29103.02021-04-16',
 '21201.02021-07-16',
 '2998.02021-07-09',
 '48327.02021-04-02',
 '29151.02020-11-20',
 '38081.02020-07-31',
 '17035.02020-12-25',
 '21109.02021-04-23',
 '48259.02020-11-06',
 '18101.02021-04-23',
 '72123.02020-10-30',
 '27125.02020-10-09',
 '4011.02021-07-16',
 '72111.02020-12-25',
 '38085.02021-01-29',
 '2050.02021-01-29',
 '47023.02020-10-16',
 '47075.02021-02-26',
 '47095.02021-04-30',
 '72083.02021-01-08',
 '13259.02021-07-23',
 '20139.02020-08-07',
 '21043.02021-10-08',
 '38033.02021-09-10',
 '42097.02020-11-06',
 '72049.02020-10-23',
 '28027.02021-03-12',
 '48467.02021-03-12',
 '40141.02021-09-17',
 '13307.02021-10-22',
 '12129.02020-10-30',
 '12077.02020-11-20',
 '13319.02021-03-12',
 '45069.02020-10-23',
 '50013.02020-10-23',
 '13049.02020-12-25',
 '54031.02021-01-22',
 '12129.02021-04-23',
 '28005.02021-01-01',
 '2997.02020-09-18',
 '38013.02020-09-04',
 '13177.02021-07-23',
 '46075.02021-09-03',
 '72135.02021-08-27',
 '28015.02020-

In [73]:
# Eliminate rows with fips_date values that do not appear in counties dataset
hospital_data_df2 = hospital_data_df2[~hospital_data_df2['fips_date'].isin(remove_list)]
hospital_data_df2

,fips_date,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,all_adult_hospital_inpatient_bed_occupied_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_pediatric_patients_hospitalized_confirmed_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum,staffed_icu_adult_patients_confirmed_covid_7_day_sum
0,10001.02020-07-31,2100.0,1939.0,1688.0,1713.0,1558.0,152.0,84.0,0.0,0.0,1876.0,322.0,203.0,266.0,189.0,28.0,28.0
1,10001.02020-08-07,2100.0,1939.0,1694.0,1672.0,1548.0,112.0,71.0,0.0,0.0,1876.0,322.0,203.0,273.0,188.0,22.0,22.0
2,10001.02020-08-14,2100.0,1939.0,1210.0,1141.0,1028.0,132.0,55.0,0.0,0.0,1340.0,198.0,134.0,183.0,128.0,29.0,29.0
3,10001.02020-08-21,2100.0,1939.0,1729.0,1650.0,1502.0,93.0,57.0,0.0,0.0,1908.0,285.0,193.0,245.0,171.0,23.0,23.0
4,10001.02020-08-28,2100.0,1939.0,1939.0,1730.0,1624.0,65.0,49.0,0.0,0.0,2100.0,315.0,203.0,251.0,182.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159866,9015.02021-09-24,951.0,937.0,783.0,565.0,565.0,56.0,42.0,0.0,0.0,783.0,119.0,119.0,34.0,34.0,14.0,10.0
159867,9015.02021-10-01,956.0,942.0,788.0,530.0,530.0,61.0,57.0,0.0,0.0,788.0,119.0,119.0,32.0,32.0,15.0,14.0
159868,9015.02021-10-08,956.0,942.0,788.0,557.0,557.0,87.0,85.0,0.0,0.0,788.0,119.0,119.0,32.0,32.0,20.0,20.0
159869,9015.02021-10-15,955.0,941.0,787.0,521.0,521.0,76.0,74.0,0.0,0.0,787.0,119.0,119.0,26.0,26.0,19.0,19.0


In [74]:
# Eliminate rows with fips_date values that do not appear in hospitals dataset
nyt_data_grouped = nyt_data_grouped[~nyt_data_grouped['fips_date'].isin(remove_list)]
nyt_data_grouped

,fips,date,cases,deaths,fips_date
0,1001.0,2020-07-31,6817,147.0,1001.02020-07-31
1,1001.0,2020-08-07,7503,150.0,1001.02020-08-07
2,1001.0,2020-08-14,8531,158.0,1001.02020-08-14
3,1001.0,2020-08-21,9009,161.0,1001.02020-08-21
4,1001.0,2020-08-28,9484,161.0,1001.02020-08-28
...,...,...,...,...,...
208912,78020.0,2021-09-24,2258,21.0,78020.02021-09-24
208913,78020.0,2021-10-01,2293,21.0,78020.02021-10-01
208914,78020.0,2021-10-08,2317,21.0,78020.02021-10-08
208915,78020.0,2021-10-15,2317,21.0,78020.02021-10-15


In [77]:
# Export grouped hospital data to CSV as hospitals_grouped.csv
hospital_data_df2.to_csv(r'C:\Users\sophc\OneDrive\Desktop\Classwork\Group-3\hospitals_grouped.csv', index=False)

In [78]:
# Export grouped NYT data to CSV as counties_grouped.csv
nyt_data_grouped.to_csv(r'C:\Users\sophc\OneDrive\Desktop\Classwork\Group-3\counties_grouped.csv', index=False)